In [1]:
class StartPoint:
    def __init__(self, x=None, y=None):        
        self.x = None
        self.y = None
        if x is not None:
            self.x = x
        else:
            self.x = 10 #int(random.random() * 200)
        if y is not None:
            self.y = y
        else:
            self.y = 10 #int(random.random() * 200)
   
    def getX(self):
        return self.x
   
    def getY(self):
        return self.y




In [2]:
import numpy as np

class Individ:
    def __init__(self, step, size, start_point, end_point, our_field):
        self.route = np.random.randint(-1,2, size=(2,size))*step  #   step*(-1) / step*1 / step*0
        self.individ_size = size
        self.fitness = 0
        self.setfitness(start_point, end_point, our_field)
        
    def setfitness(self, start_point, end_point, our_field):   #start_point = (x,y)
        cumsum = np.cumsum(self.route, axis=1) + np.array(start_point).reshape((2,1))
        index = self.checkblocks(cumsum, our_field)
        #print ('cumsum ', cumsum[:, index])
        distance = np.linalg.norm(cumsum[:, index] - np.array(end_point))      
        self.fitness = distance    # check normalization   
        #print('fitness individ ', self.fitness)
        
    def getfitness(self):
        return self.fitness
    
    def checkblocks(self, cumsum, our_field):
        for i in range(1, self.individ_size):
            if (any(cumsum[:, i] < 0) or (cumsum[0, i] >= our_field.field_size[0]) or (cumsum[1, i] >= our_field.field_size[1])):
                return i-1 
            elif our_field.getvalue(cumsum[0, i], cumsum[1, i]):
                return i-1
        return self.individ_size-1
    
        

In [3]:
class Field:
    def __init__(self, size):
        self.matrix = np.zeros(size)
        self.matrix[0, :] = 1
        self.matrix[:, 0] = 1
        self.matrix[:, size[1]-1] = 1
        self.matrix[size[0]-1, 0] = 1
        self.field_size = size
    
    def addSquareBlock(self, left_top, right_bottom):  #point = (x,y)
        for i in range(left_top[0], right_bottom[0]):
            for j in range(left_top[1], right_bottom[1]):
                self.matrix[i, j] = 1
    
    # add diff shapes of blocks
        
    def getmatrix(self):
        return self.matrix
    
    def getvalue(self,x,y):
        return self.matrix[x,y]
    
        
        

In [4]:
class Population:
    def __init__(self, step, individSize, populationSize, start_point, end_point, our_field):
        self.population = []
        self.fitness = []
        for i in range(0, populationSize):
            newInd = Individ(step, individSize, start_point, end_point, our_field)
            self.population.append(newInd)            
            self.fitness.append(newInd.fitness)
                
    def getFitness(self):
        return self.fitness
    
    def getBest(self):
        #print('fitness ', self.fitness)
        #print('population ', self.population)
        sortedPopulation = [pop for _,pop in sorted(zip(self.fitness, self.population), key = lambda x: x[0])]
        
        #self.fitness.sort(reverse=True)
        
        
        return sortedPopulation[0]
    
    def add_individ(self, newIndivid):
        self.population.append(newIndivid)
        self.fitness.append(newIndivid.fitness)
    
        
    
    

In [7]:
import random

class GA:
    def __init__(self, user_mutationRate, user_crossoverProbability, user_elitism, start_point, end_point, our_field): #, tourmanager):
        #self.tourmanager = tourmanager
        self.mutationRate = user_mutationRate
        #self.tournamentSize = 10
        self. crossoverProbability = user_crossoverProbability
        self.elitism = user_elitism
        self.start_point = start_point
        self.end_point = end_point
        self.our_field = our_field
        
    def CreateFirstPopulation(self, user_step, user_individSize, user_populationSize):
        initialPop = Population(step=user_step, individSize=user_individSize, populationSize=user_populationSize, 
                                start_point=self.start_point, end_point=self.end_point, our_field=self.our_field)
        return initialPop
        
    def chooseParents(self, newPopulation):
        #print ('parent fitness ', newPopulation.fitness)
        reversed_fitness = 1.0/np.array(newPopulation.fitness)
        select_prob = reversed_fitness/sum(reversed_fitness)
        #add tournamentsize
        selected = np.random.choice(newPopulation.population, 2, p=select_prob, replace=False)
        return list(selected)
    
    def crossover(self, parents):
        startPos = random.randint(0,parents[0].individ_size)
        endPos = random.randint(startPos, parents[0].individ_size)   
        pp = random.random()
        if pp <= self.crossoverProbability:
            
            parents[0].route[:, startPos:endPos], parents[1].route[:, startPos:endPos] = parents[1].route[:, startPos:endPos], parents[0].route[:, startPos:endPos]
            
        return parents
    
    def mutate(self, children):
        if random.random() < self.mutationRate:
            indx = random.randint(0, children[0].individ_size-1)
            children[0].route[0, indx], children[0].route[1, indx] = children[0].route[1, indx], children[0].route[0, indx]
        if random.random() < self.mutationRate:
            indx = random.randint(0, children[1].individ_size-1)
            children[1].route[0, indx], children[1].route[1, indx] = children[1].route[1, indx], children[1].route[0, indx]
        return children
    
    def fittest(self, child_list, n): # n - how much return
        child_list.sort(key = lambda x: x.fitness)
        return child_list[:n]
              
        
    def evolve(self, step, individSize, populationSize, generation):  #generation - previous population
        
        children_list = []
        for i in range(populationSize):
            #choose parents
            #print('generation fitness ', generation.fitness)
            parents = self.chooseParents(generation)  
            #make 2 children
            children = self.crossover(parents)
            #mutate
            children = self.mutate(children)
            children[0].setfitness(self.start_point, self.end_point, self.our_field)
            children[1].setfitness(self.start_point, self.end_point, self.our_field)
            #print('children list ', children_list)
            #print('children ', type(children), len(children))
            children_list = children_list + children
            
        #Create new generation with elitism?
        
        newPopulation = Population(step, individSize, populationSize, self.start_point, self.end_point, self.our_field)
        if self.elitism == True:
            newPopulation.add_individ(generation.getBest())
            best_children = self.fittest(children_list, populationSize-1)
        else:
            best_children = self.fittest(children_list, populationSize)
        
        for child in best_children:
            newPopulation.add_individ(child)
            
        #print('new population ', newPopulation)
                  
        return newPopulation
        

In [9]:
# common parameters for all users
gameField = Field(size = (30, 30))
startpoint = (0,0)
endpoint = (20,20)

# parameters of one user
mutationRate=0.1
crossoverProbability=0.8
elitism=True
step=2
individSize=50
populationSize=100

ga = GA(user_mutationRate=mutationRate, user_crossoverProbability=crossoverProbability, user_elitism=elitism, 
        start_point=startpoint, end_point=endpoint, our_field=gameField)
initialPop = ga.CreateFirstPopulation(user_step=step, user_individSize=individSize, user_populationSize=populationSize)

print('evolve function: ')
while not any(np.array(initialPop.fitness) < 2):
    initialPop = ga.evolve(step = step, individSize = individSize, populationSize = populationSize, generation = initialPop)
    print('min fitness ', min(initialPop.fitness))



evolve function: 
min fitness  18.4390889146
min fitness  12.6491106407
min fitness  8.24621125124
min fitness  8.24621125124
min fitness  12.8062484749
min fitness  10.0
min fitness  5.65685424949
min fitness  6.32455532034
min fitness  5.65685424949
min fitness  8.24621125124
min fitness  18.973665961
min fitness  21.6333076528
min fitness  18.973665961
min fitness  12.8062484749
min fitness  18.0
min fitness  20.591260282
min fitness  18.4390889146
min fitness  14.1421356237
min fitness  11.6619037897
min fitness  6.32455532034
min fitness  6.32455532034
min fitness  2.0
min fitness  5.65685424949
min fitness  19.6977156036
min fitness  7.21110255093
min fitness  8.24621125124
min fitness  6.32455532034
min fitness  10.0
min fitness  18.1107702763
min fitness  18.1107702763
min fitness  18.1107702763
min fitness  2.82842712475
min fitness  4.472135955
min fitness  8.94427191
min fitness  18.973665961
min fitness  6.32455532034
min fitness  10.1980390272
min fitness  12.6491106407
mi